In [ ]:
import locale
import os
locale.getpreferredencoding = lambda: "UTF-8"

hugging_face_token = os.environ.get("HF_TOKEN")

In [ ]:
!pip install -q gdown pypdf
!pip install -q accelerate bitsandbytes
!pip install -q transformers
!pip install -q langchain
!pip install -q sentence-transformers
!pip install -q cohere llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) .

In [ ]:
import os
import torch
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import bitsandbytes as bnb

In [ ]:
hugging_face_model_id = "meta-llama/Llama-2-13b-hf"
cache_dir = "./model/"

!mkdir "index"
index_persist_dir = "./index"

environment = "COLAB" # either COLAB or KAGGLE

if environment == "KAGGLE":
  dental_corpus_zip_path = "/kaggle/working/dental-guideline-dataset.zip"
  index_zip_path = "/kaggle/working/dental-guidelines-index.zip"
else:
  dental_corpus_zip_path = "./dental-guideline-dataset.zip"
  index_zip_path = "./dental-guidelines-index.zip"

In [ ]:
login(token=hugging_face_token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import zipfile

""" Only one of the following is needed """
""" If the Dataset Zip file is downloaded then the indexes will be build (again!) """
""" If the Dataset Zip file is not downloaded then make sure you download the indexes """

# Download Dataset Zip and unzip
!gdown 1izm1q-HXwgPhNICZmKLnis7JkU9Co79L
with zipfile.ZipFile(dental_corpus_zip_path, 'r') as zip_ref: zip_ref.extractall("./")

# Download Index Zip and unzip
!gdown 1l3aFkVeW9hn2d0Mb9QFVxTy0hrZd8BNG
with zipfile.ZipFile(index_zip_path, 'r') as zip_ref: zip_ref.extractall("./")

Downloading...
From: https://drive.google.com/uc?id=1izm1q-HXwgPhNICZmKLnis7JkU9Co79L
To: /content/dental-guideline-dataset.zip
100% 670M/670M [00:09<00:00, 69.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1l3aFkVeW9hn2d0Mb9QFVxTy0hrZd8BNG
To: /content/dental-guidelines-index.zip
100% 103M/103M [00:01<00:00, 67.3MB/s] 


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(hugging_face_model_id, cache_dir=cache_dir, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(hugging_face_model_id, cache_dir=cache_dir, device_map="auto", quantization_config=bnb_config)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def prompt(prompt, model, streamer):
  model_inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
  model_output = model.generate(**model_inputs, streamer=streamer, use_cache=True, max_new_tokens=float('inf'))
  model_output_text = tokenizer.decode(model_output[0][model_inputs.input_ids.shape[-1]+1:], skip_special_tokens=True)
  return model_output_text

In [ ]:
prompt_text = """<<SYS>>
You work as a question generator who's task is to generate questions as instructed from a text corpus. The quality of the questions are high quality and the theme is not general questions but secific questions.
<</SYS>>

[INST]Your task is to generate 2 questions, which are not general but specific to informations, based on the text denoted in tripple backtics. You have to respond in the following format denoted in tripple quotation: \"\"\"question list:\n 1. ...?\n2. ...?\"\"\"[/INST]

```
CHUNKS_TEXT_HERE
```
"""

In [ ]:
response = prompt(prompt_text, model, streamer)

In [ ]:
str(response)